# Astro Tools

Here we write definitions and test functions used for the project 

## Imports

In [1]:
import numpy as np
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import matplotlib.cm as mcm
import sys 

%matplotlib inline

## Definitions

### Vaccuum rest wavelengths of spectral lines used here

In [15]:
l_ha = 6564.614 *u.Angstrom
l_hb = 4862.721 *u.Angstrom
l_O3_4960 = 4960.295 *u.Angstrom
l_O3_5008 = 5008.239 *u.Angstrom
l_O2_3727 = 3727.092 *u.Angstrom
l_O2_3729 = 3729.875 *u.Angstrom

l_NII_6549 = 6549.86 *u.Angstrom
l_NII_6585 = 6585.27 *u.Angstrom
l_SII_6718 = 6718.29 * u.Angstrom
l_SII_6732 = 6732.68 *u.Angstrom
l_OI_6302 = 6302.046 *u.Angstrom

## Functions

### Cardelli dust correction


The extinction  law is given by:  
$\frac{A_\lambda}{A_V} = a(x) + \frac{b(x)}{R_v}$  
Where we assume that $R_v=3.1$  

We also have $k(\lambda)=R_V\,\frac{A_\lambda}{A_V} = R_v \cdot a(x) + b(x)$. 

Observed fluxed can be corrected for dust attenuation using:  
$f_{\lambda,int} = f_{\lambda,obs} \cdot 10^{0.4*k(\lambda)}$

**From Cardelli et al. 1989:**  
$x  \equiv 1/\lambda$, and with $y  \equiv (x - 1.82)$, for $x \in$ [1.1 $\mu$m$^{-1}$ -3.3 $\mu$m$^{-1}$]:   
$a(x) = 1 + 0.17699\,y - 0.50447\,y^2 - 0.02427\,y^3 + 0.72085\,y^4+ 0.01979\,y^5 - 0.77530\,y^6 + 0.32999\,y^7 $  
$b(x) = 1.41338\,y + 2.28305\,y^2 + 1.07233\,y^3 - 5.38434\,y^4 - 0.62251\,y^5 + 5.30260\,y^6 - 2.09002\,y^7 $


In [3]:
def k_Cardelli(l,Rv=3.1):
  try:
    l=l.to(u.micrometer)
  except:
    print('Wavelength unit invalid')
  if (l<1/1.1*u.micrometer) & (l>1/3.3*u.micrometer):
    x = 1/l.value
    y = x-1.82
    a = 1 + 0.17699*y - 0.50447*y**2 - 0.02427*y**3 + 0.72085*y**4+ 0.01979*y**5 - 0.77530*y**6 + 0.32999*y**7
    b = 1.41338*y + 2.28305*y**2 + 1.07233*y**3 - 5.38434*y**4 - 0.62251*y**5 + 5.30260*y**6 - 2.09002*y**7
    k = Rv * a + b
  else:
    print('Wavelength outside the optical range, function not adapted.')
  return(k)

In [4]:
l_Ha =  6564.6 *u.Angstrom # Halpha vacuum wavelength 
k_Ha = k_Cardelli(l_Ha)
print(r'$k(\lambda)$ at H$\alpha$ wavelength:', k_Ha)

l_Hb =  4862.721 *u.Angstrom # Hbeta vacuum wavelength 
k_Hb = k_Cardelli(l_Hb)
print(r'$k(\lambda)$ at H$\beta$ wavelength:', k_Hb)

$k(\lambda)$ at H$\alpha$ wavelength: 2.5342494600750585
$k(\lambda)$ at H$\beta$ wavelength: 3.607546489739622


### Calculate the color excess based on Halpa+ Hbta fluxes

$f_{int} = f_{obs} \cdot 10^{0.4 \cdot E(B-V)\cdot k(\lambda)}$  
So,using:  
$f_{H\alpha,int} = f_{H\alpha,obs} \cdot 10^{0.4 \cdot E(B-V)\cdot k(H\alpha)}$ and  
$f_{H\beta,int} = f_{H\beta,obs} \cdot 10^{0.4 \cdot E(B-V)\cdot k(H\beta)}$  
One gets to:

$$E(B-V) = \frac{-2.5}{k(H\alpha)-k(H\beta)} * \log(\frac{fH\alpha/fH\beta_{obs}}{fH\alpha/fH\beta_{int}})$$  

From Osterbrock+2006 (2nd edition), looking at Case B recombination lines in the low-density limit i.e. $n_e< 10^4 \rm{cm}^{-3}$ and assuming T=1e4K (see Table 4.2, p.73), one finds that: $\frac{fH\alpha}{fH\beta}_{int}=2.87$.



In [5]:
def EBV(fHa,fHb,ha_hb_int=2.87):
  # Intrinsic Halpha to Hbeta ratio ha_hb_int=2.86 obtained for ne=100cm-3 and T=1e4K
  l_Ha =  6564.6 *u.Angstrom # Halpha vacuum wavelength 
  l_Hb =  4862.721 *u.Angstrom # Hbeta vacuum wavelength 
  ebv = -2.5/(k_Cardelli(l_Ha)-k_Cardelli(l_Hb)) * np.log10(fHa/fHb/ha_hb_int)
  return(ebv)


### Operate dust correction

$f_{int} = f_{obs} \cdot 10^{0.4 \cdot E(B-V)\cdot k(\lambda)}$

In [6]:
def dust_corr_factor(l,ebv):
  kl =  k_Cardelli(l)
  return(10**(0.4*ebv*kl))

In [7]:
print(dust_corr_factor(l_Ha,ebv=0.3))
print(dust_corr_factor(l_Hb,ebv=0.3))

2.0142340588222347
2.709602465264213


### O32 definitions according to different reference papers

In [8]:
def O32_I18(O3_5007,O2_3727):
  # Definition given in Izotov et al. 2018
  return(O3_5007/O2_3727) 

def O32_F22(O3_5007,O2_3726,O2_3729):
  # Definition given in Flury et al. 2022
  return(O3_5007/(O2_3726+O2_3729)) 

def O32_P22(O3_4959,O3_5008,O2_3726,O2_3729):
  # Definition given in Papovich et al. 2022
  return((O3_4959+O3_5008)/(O2_3726+O2_3729)) 

### BPT selection

#### Kewley+2001 definition (Optical)

BPT diagram selection requires [OIII], H$\beta$, H$\alpha$ and [NII] ([N II]$_{\lambda\lambda 6548,84}$, I think),[SII] or [OI].  
Starburst galaxies satisfy the following criteria:  
$$\log_{10}\left(\frac{[\rm{OIII}]_{\lambda 5007}}{\rm{H\beta}}\right)<\frac{0.61}{\log_{10}([\rm{NII}]/\rm{H\alpha})-0.47}+1.19$$
$$\log_{10}\left(\frac{[\rm{OIII}]_{\lambda 5007}}{\rm{H\beta}}\right)<\frac{0.72}{\log_{10}([\rm{SII}]_{\lambda\lambda 6717,31}/\rm{H\alpha})-0.32}+1.30$$
$$\log_{10}\left(\frac{[\rm{OIII}]_{\lambda 5007}}{\rm{H\beta}}\right)<\frac{0.73}{\log_{10}([\rm{OI}]_{\lambda 6300}/\rm{H\alpha})+0.59}+1.33$$


In [12]:
def K01_1(OIII_5007,Hb,NII_6548,NII_6584,Ha):
  K01_left = np.log10(OIII_5007/Hb)
  K01_right_1 = 0.61/(np.log10((NII_6548,NII_6584)/Ha)-0.47) + 1.19
  starburst_1 = np.where((K01_left<K01_right_1)&(np.log10((NII_6548,NII_6584)/Ha)<0.47),1,0)
  return(starburst_1)

def K01_2(OIII_5007,Hb,SII_6717,SII_6731,Ha):
  K01_left = np.log10(OIII_5007/Hb)
  K01_right_2 = 0.72/(np.log10((SII_6717+SII_6731)/Ha)-0.32) + 1.30
  starburst_2 = np.where((K01_left<K01_right_2)&(np.log10((SII_6717+SII_6731)/Ha)<0.32),1,0)
  return(starburst_2)

def K01_3(OIII_5007,Hb,OI_6300,Ha):
  K01_left = np.log10(OIII_5007/Hb)
  K01_right_3 = 0.73/(np.log10(OI_6300/Ha)+0.59) + 1.33
  starburst_3 = np.where((K01_left<K01_right_3)&(np.log10(OI_6300/Ha)<-0.59),1,0)
  return(starburst_3)
  

#### Kauffmann+2003 definition (Optical)

BPT diagram selection requires [OIII], H$\beta$, H$\alpha$ and [NII].  
Starburst galaxies satisfy the following criteria: 
$$\log_{10}\left(\frac{[\rm{OIII}]_{\lambda 5007}}{\rm{H\beta}}\right)<\frac{0.61}{\log_{10}([\rm{NII}]_{\lambda 6583}/\rm{H\alpha})-0.05}+1.3$$


In [2]:
def K03(OIII_5007,Hb,NII_6583,Ha):
  K03_left = np.log10(OIII_5007/Hb)
  K03_right = 0.61/(np.log10(NII_6583/Ha)-0.05) + 1.3
  starburst = np.where((K03_left<K03_right)&(np.log10(NII_6583/Ha)<0.05),1,0) 
  #second condition to avoid going outside the definition range of the function
  return(starburst)

#### Blecha+2018 definition (IR)

Need WISE magnitudes in bands W1, W2 and W3 (centered on 3.4, 4.6, and 12 $\mu$m) in Vega magnitudes :  
$$[W1-W2]<0.5\ \ \rm{or} \ \ [W2-W3]<2.2 \ \ \rm{or} \ \ [W1-W2] < 2\cdot [W2 - W3] - 8.9 $$